# Tarefa 02

- Leia os enunciados com atenção
- Saiba que pode haver mais de uma resposta correta
- Insira novas células de código sempre que achar necessário
- Em caso de dúvidas, procure os monitores
- Divirta-se :)

#### 1. Carregue os dados da tarefa anterior

In [7]:
from pandas_datareader import data as web
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import yfinance as yf

#### 2. Média móvel

1. Faça uma média móvel em 90 dias dos valores de fechamento ajustados dos ativos que você escolheu
2. Utilizando a fução ```rolling()```, calcule o desvio pardão desses ativos ao longo do tempo, utilizando a mesma janela móvel do item acima
2. Monte um gráfico de linhas para as médias móveis dos ativos da sua base
4. Monte um gráfico de linhas para os desvios padrão em janela móvel
5. Compare os dois gráficos que você fez com o da tarefa anterior. Veja se eles apontam conclusões semelhantes quanto aos períodos de maior volatilidade (variação) dos ativos.

In [15]:
ativos = ['^BVSP', 'TSLA', 'PETR4.SA', 'ABEV3.SA', 'VALE3.SA']
dt_ini = '2020-01-01'

df = yf.download(ativos, start=dt_ini)['Adj Close']
df.head()

[*********************100%%**********************]  5 of 5 completed


Ticker,ABEV3.SA,PETR4.SA,TSLA,VALE3.SA,^BVSP
Date,,,,,
2020-01-02,16.175152,11.240597,28.684000,36.413361,118573.0
2020-01-03,15.947686,11.149062,29.534000,36.145267,117707.0
2020-01-06,16.023506,11.280874,30.102667,35.930794,116878.0
2020-01-07,16.057203,11.236935,31.270666,36.192184,116662.0
2020-01-08,15.981384,11.167367,32.809334,36.198879,116247.0


In [33]:
# Transformar os dados utilizando stack e reset_index
stacked_data = df.stack().reset_index()
stacked_data.columns = ['Data', 'Ativo', 'Valor']

In [34]:
# Calcular a média móvel de 90 dias para cada ativo
stacked_data['90D_MA'] = stacked_data.groupby('Ativo')['Valor'].transform(lambda x: x.rolling(window=90).mean())

# Exibir os primeiros registros do DataFrame resultante
print(stacked_data.head(), '\n')

# Exibir os últimos registros do DataFrame resultante para ver as médias móveis calculadas
print(stacked_data.tail())

        Data     Ativo          Valor  90D_MA
0 2020-01-02  ABEV3.SA      16.175152     NaN
1 2020-01-02  PETR4.SA      11.240597     NaN
2 2020-01-02      TSLA      28.684000     NaN
3 2020-01-02  VALE3.SA      36.413361     NaN
4 2020-01-02     ^BVSP  118573.000000     NaN 

           Data     Ativo          Valor         90D_MA
5529 2024-06-12  ABEV3.SA      11.310000      12.367333
5530 2024-06-12  PETR4.SA      36.200001      37.512410
5531 2024-06-12      TSLA     176.725006     177.450500
5532 2024-06-12  VALE3.SA      60.500000      62.911286
5533 2024-06-12     ^BVSP  120153.929688  126958.676997


In [39]:
# Calcular o desvio padrão de 90 dias para cada ativo
stacked_data['90D_STD'] = stacked_data.groupby('Ativo')['Valor'].transform(lambda x: x.rolling(window=90).std())

# Exibir os primeiros registros do DataFrame resultante
print(stacked_data.head(), '\n')

# Exibir os últimos registros do DataFrame resultante para ver as médias móveis e desvios padrão calculados
print(stacked_data.tail())

        Data     Ativo          Valor  90D_MA  90D_STD
0 2020-01-02  ABEV3.SA      16.175152     NaN      NaN
1 2020-01-02  PETR4.SA      11.240597     NaN      NaN
2 2020-01-02      TSLA      28.684000     NaN      NaN
3 2020-01-02  VALE3.SA      36.413361     NaN      NaN
4 2020-01-02     ^BVSP  118573.000000     NaN      NaN 

           Data     Ativo          Valor         90D_MA      90D_STD
5529 2024-06-12  ABEV3.SA      11.310000      12.367333     0.476081
5530 2024-06-12  PETR4.SA      36.200001      37.512410     2.179033
5531 2024-06-12      TSLA     176.725006     177.450500    12.317404
5532 2024-06-12  VALE3.SA      60.500000      62.911286     1.619724
5533 2024-06-12     ^BVSP  120153.929688  126958.676997  2410.761562


In [35]:
# Gráfico de linha
# Renomear as colunas para garantir que estejam corretas
stacked_data.rename(columns={'Date': 'Data', 'Ticker': 'Ativo', 'Adj Close': 'Valor'}, inplace=True)

# Criar o gráfico de linhas com Plotly
fig = px.line(stacked_data, x='Data', y='90D_MA', color='Ativo', title='Médias Móveis de 90 Dias dos Ativos', labels={'90D_MA': 'Média Móvel (90 Dias)', 'Data': 'Data', 'Ativo': 'Ativo'})
fig.update_layout(template='plotly_dark')

# Mostrar o gráfico
fig.show()

In [40]:
# Criar o gráfico de linhas com Plotly para desvio padrão
fig = px.line(stacked_data, x='Data', y='90D_STD', color='Ativo', title='Desvios Padrão de 90 Dias dos Ativos', labels={'90D_STD': 'Desvio Padrão (90 Dias)', 'Data': 'Data', 'Ativo': 'Ativo'})
fig.update_layout(template='plotly_dark')

# Mostrar o gráfico
fig.show()

Compare os dois gráficos que você fez com o da tarefa anterior. Veja se eles apontam conclusões semelhantes quanto aos períodos de maior volatilidade (variação) dos ativos.

#### 3. Gráfico de *candlestick*
1. Selecione o período dos últimos 60 dias (corridos) para um dos ativos da sua base
2. Monte um gráfico de *candlestick* para esses dados.

In [44]:
# Obtendo os dados de mercado para o ativo TSLA
data_tsla = yf.download('TSLA', start='2020-01-01', end='2024-01-01')

# Selecionando os últimos 60 dias corridos
ultimo_dia = data_tsla.index[-1]
data_60d_corridos = data_tsla.loc[ultimo_dia - pd.Timedelta(days=59):ultimo_dia]

print(data_60d_corridos)

[*********************100%%**********************]  1 of 1 completed

                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2023-10-31  196.119995  202.800003  194.070007  200.839996  200.839996   
2023-11-01  204.039993  205.990005  197.850006  205.660004  205.660004   
2023-11-02  212.970001  219.199997  211.449997  218.509995  218.509995   
2023-11-03  221.149994  226.369995  218.399994  219.960007  219.960007   
2023-11-06  223.979996  226.320007  215.000000  219.270004  219.270004   
2023-11-07  219.979996  223.119995  215.720001  222.179993  222.179993   
2023-11-08  223.149994  224.149994  217.639999  222.110001  222.110001   
2023-11-09  219.750000  220.800003  206.679993  209.979996  209.979996   
2023-11-10  210.029999  215.380005  205.690002  214.649994  214.649994   
2023-11-13  215.600006  225.399994  211.610001  223.710007  223.710007   
2023-11-14  235.029999  238.139999  230.720001  237.410004  237.410004   
2023-11-15  239.289993  246.699997  23

In [45]:
# Criando o gráfico de candlestick
fig = go.Figure(data=[go.Candlestick(x=data_tsla.index,
                open=data_tsla['Open'],
                high=data_tsla['High'],
                low=data_tsla['Low'],
                close=data_tsla['Close'])])

# Personalizando o layout do gráfico
fig.update_layout(title='Candlestick dos Últimos 60 Dias Corridos do Ativo TSLA',
                  xaxis_title='Data',
                  yaxis_title='Preço',
                  template='plotly_dark')

# Mostrando o gráfico
fig.show()